In [1]:
import pandas as pd

df = pd.read_csv( r"C:\Users\viren\OneDrive\Desktop\Road accident data analytics\archive (6)\clean_accidents.csv")
df.head()


,State Name,City Name,Year,Month,Day of Week,Time of Day,Accident Severity,Number of Vehicles Involved,Vehicle Type Involved,Number of Casualties,...,Road Type,Road Condition,Lighting Conditions,Traffic Control Presence,Speed Limit (km/h),Driver Age,Driver Gender,Driver License Status,Alcohol Involvement,Accident Location Details
0,Jammu and Kashmir,Unknown,2021,5,Monday,1:46,Serious,5,Cycle,0,...,National Highway,Wet,Dark,Signs,61,66,Male,Nan,Yes,Curve
1,Uttar Pradesh,Lucknow,2018,1,Wednesday,21:30,Minor,5,Truck,5,...,Urban Road,Dry,Dusk,Signs,92,60,Male,Nan,Yes,Straight Road
2,Chhattisgarh,Unknown,2023,5,Wednesday,5:37,Minor,5,Pedestrian,6,...,National Highway,Under Construction,Dawn,Signs,120,26,Female,Nan,No,Bridge
3,Uttar Pradesh,Lucknow,2020,6,Saturday,0:31,Minor,3,Bus,10,...,State Highway,Dry,Dark,Signals,76,34,Female,Valid,Yes,Straight Road
4,Sikkim,Unknown,2021,8,Thursday,11:21,Minor,5,Cycle,7,...,Urban Road,Wet,Dusk,Signs,115,30,Male,Nan,No,Intersection


In [3]:
df.columns


Index(['State Name', 'City Name', 'Year', 'Month', 'Day of Week',
       'Time of Day', 'Accident Severity', 'Number of Vehicles Involved',
       'Vehicle Type Involved', 'Number of Casualties', 'Number of Fatalities',
       'Weather Conditions', 'Road Type', 'Road Condition',
       'Lighting Conditions', 'Traffic Control Presence', 'Speed Limit (km/h)',
       'Driver Age', 'Driver Gender', 'Driver License Status',
       'Alcohol Involvement', 'Accident Location Details'],
      dtype='object')

In [7]:
df.columns


Index(['State Name', 'City Name', 'Year', 'Month', 'Day of Week',
       'Time of Day', 'Accident Severity', 'Number of Vehicles Involved',
       'Vehicle Type Involved', 'Number of Casualties', 'Number of Fatalities',
       'Weather Conditions', 'Road Type', 'Road Condition',
       'Lighting Conditions', 'Traffic Control Presence', 'Speed Limit (km/h)',
       'Driver Age', 'Driver Gender', 'Driver License Status',
       'Alcohol Involvement', 'Accident Location Details'],
      dtype='object')

In [9]:
# ===== Heading: Robust Month -> Month Number conversion, Year fix, and Date creation =====

import pandas as pd

# assume df already loaded
# 1) Trim whitespace and make a string copy of Month
m = df['Month'].astype(str).str.strip()

# 2) Try to parse numeric months (e.g. "5", "05")
m_num = pd.to_numeric(m, errors='coerce')               # will give NaN for names like "May"

# 3) Try to parse full month names (e.g. "May", "January")
m_name_full = pd.to_datetime(m, format='%B', errors='coerce').dt.month

# 4) Try to parse abbreviated month names (e.g. "May", "Jan", "Feb")
m_name_abbr = pd.to_datetime(m, format='%b', errors='coerce').dt.month

# 5) Combine the results: prefer numeric, then full name, then abbreviation
month_combined = m_num.fillna(m_name_full).fillna(m_name_abbr)

# 6) If still NaN, try a final generic parse (mixed formats)
month_generic = pd.to_datetime(m, errors='coerce').dt.month
month_combined = month_combined.fillna(month_generic)

# 7) Assign back to df as integers (if any remaining NaN we'll show them)
df['Month'] = month_combined

# 8) Fix Year column safely (convert to numeric)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# 9) Report any rows where Month or Year could not be parsed
bad_rows = df[df['Month'].isna() | df['Year'].isna()]
if len(bad_rows) > 0:
    print("⚠️ Rows with missing/invalid Month or Year (showing up to 10):")
    display(bad_rows.head(10))
else:
    print("✅ All Month and Year values parsed successfully.")

# 10) Drop rows with missing Year or Month (optional - recommened if only a few bad rows)
# If you prefer to keep them, comment out the next line.
df = df.dropna(subset=['Year', 'Month']).copy()

# 11) Convert Month and Year to integer type
df['Month'] = df['Month'].astype(int)
df['Year']  = df['Year'].astype(int)

# 12) Create Date column = Year + Month + Day(1)
df['Date'] = pd.to_datetime({'year': df['Year'], 'month': df['Month'], 'day': 1})

# 13) Quick check
print("\nSample Date values:")
display(df[['Year','Month','Date']].head())

print("\nData types now:")
print(df.dtypes[['Year','Month','Date']])


✅ All Month and Year values parsed successfully.

Sample Date values:


C:\Users\viren\AppData\Local\Temp\ipykernel_9060\3664158541.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  month_generic = pd.to_datetime(m, errors='coerce').dt.month


,Year,Month,Date
0,2021,5,2021-05-01
1,2018,1,2018-01-01
2,2023,5,2023-05-01
3,2020,6,2020-06-01
4,2021,8,2021-08-01



Data types now:
Year              int64
Month             int64
Date     datetime64[ns]
dtype: object


In [14]:
bi_df = df[[
    'State Name','City Name','Year','Month','Date','Day of Week','Time of Day',
    'Accident Severity','Number of Vehicles Involved','Vehicle Type Involved',
    'Number of Casualties','Number of Fatalities','Weather Conditions','Road Type',
    'Road Condition','Lighting Conditions','Traffic Control Presence',
    'Speed Limit (km/h)','Driver Age','Driver Gender','Driver License Status',
    'Alcohol Involvement'
]]

bi_df.to_csv("../Road accident data analytics/accidents_for_powerbi.csv", index=False)
print("Power BI file saved!")


Power BI file saved!


In [15]:
import os
os.getcwd()


'C:\\Users\\viren\\OneDrive\\Desktop\\jupyter notebook'

In [16]:
import os

print(os.listdir("../Road accident data analytics"))


['accidents_for_powerbi.csv', 'archive (6)', 'archive (6).zip']


In [17]:
import os
os.path.abspath("../Road accident data analytics/accidents_for_powerbi.csv")


'C:\\Users\\viren\\OneDrive\\Desktop\\Road accident data analytics\\accidents_for_powerbi.csv'